In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
import tensorflow as tf

# prepare data

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [ ]:
unlabeld_index = np.ones(y_train.shape, np.bool)

N = 20
for i in range(10):
  idx = np.where(y_train == i)[0][:N]
  unlabeld_index[idx] = 0

In [ ]:
x_unlabeld = x_train[np.where(unlabeld_index)[0], ...]

x_train = x_train[np.where(~unlabeld_index)[0], ...]
y_train = y_train[np.where(~unlabeld_index)[0], ...]

In [ ]:
# examples of categorical crossentropy
cce = keras.losses.CategoricalCrossentropy()

# a labeled data from the second class
y_true = [[0, 1, 0, 0]]
y_pred = [[0.05, 0.95, 0, 0]]
print(cce(y_true, y_pred).numpy())

# an ulabeled data
y_true = [[0, 0, 0, 0]]
y_pred = [[0.05, 0.95, 0, 0]]
print(cce(y_true, y_pred).numpy())

# another ulabeled data
y_true = [[0, 0, 0, 0]]
y_pred = [[0.1, 0.4, 0.3, 0.2]]
print(cce(y_true, y_pred).numpy())

0.051293306
0.0
0.0


In [ ]:
# Convert class vectors to binary class matrices.
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# from keras.callbacks import EarlyStopping
# es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)

In [ ]:
# define model
regularizer=keras.regularizers.l2(1e-3)
input = keras.layers.Input(shape=x_train[0].shape)

c1 = keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='elu', kernel_regularizer=regularizer)(input)
c2 = keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='elu', kernel_regularizer=regularizer)(c1)
m1 = keras.layers.MaxPooling2D()(c2)
c3 = keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='elu', kernel_regularizer=regularizer)(m1)
c4 = keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='elu', kernel_regularizer=regularizer)(c3)
m2 = keras.layers.MaxPooling2D()(c4)
f = keras.layers.Flatten(input_shape=x_train[0].shape)(m2)
dropout = keras.layers.Dropout(0.5)(f)
d1 = keras.layers.Dense(units=512, activation='elu', kernel_regularizer=regularizer)(dropout)
output = keras.layers.Dense(units=num_classes, activation='softmax', kernel_regularizer=regularizer)(d1)

model = keras.Model(input, output)
model.summary()

Model: "model_59"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_42 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_164 (Conv2D)          (None, 30, 30, 128)       3584      
_________________________________________________________________
conv2d_165 (Conv2D)          (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d_82 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_166 (Conv2D)          (None, 12, 12, 128)       147584    
_________________________________________________________________
conv2d_167 (Conv2D)          (None, 10, 10, 128)       147584    
_________________________________________________________________
max_pooling2d_83 (MaxPooling (None, 5, 5, 128)         0  

In [ ]:
# compile model

optimizer = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# train model
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=10,
                    validation_data=(x_test, y_test),
                    
                    )

Epoch 1/10
4/4 [==============================] - 3s 748ms/step - loss: 32.2424 - accuracy: 0.0942 - val_loss: 23.5384 - val_accuracy: 0.1000
Epoch 2/10
4/4 [==============================] - 2s 636ms/step - loss: 22.2260 - accuracy: 0.1184 - val_loss: 15.5483 - val_accuracy: 0.1146
Epoch 3/10
4/4 [==============================] - 2s 632ms/step - loss: 13.9587 - accuracy: 0.1343 - val_loss: 12.1004 - val_accuracy: 0.1216
Epoch 4/10
4/4 [==============================] - 2s 639ms/step - loss: 11.0102 - accuracy: 0.0907 - val_loss: 4.8111 - val_accuracy: 0.1312
Epoch 5/10
4/4 [==============================] - 2s 631ms/step - loss: 5.0795 - accuracy: 0.1272 - val_loss: 4.4403 - val_accuracy: 0.1058
Epoch 6/10
4/4 [==============================] - 2s 638ms/step - loss: 4.4212 - accuracy: 0.1461 - val_loss: 3.9626 - val_accuracy: 0.1244
Epoch 7/10
4/4 [==============================] - 2s 630ms/step - loss: 4.1020 - accuracy: 0.1743 - val_loss: 3.8152 - val_accuracy: 0.1311
Epoch 8/10
4/

In [ ]:
new_x_train = []
new_y_train = []
for i in range(len(x_unlabeld)):
  new_x_train.append(x_unlabeld[i])
  new_y_train.append(0)
  rot90 = tf.image.rot90(x_unlabeld[i])
  new_x_train.append(rot90)
  new_y_train.append(1)
  rot180 = tf.image.rot90(x_unlabeld[i], k=2)
  new_x_train.append(rot180)
  new_y_train.append(2)
  rot270 = tf.image.rot90(x_unlabeld[i], k=3)
  new_x_train.append(rot270)
  new_y_train.append(3)

In [ ]:
new_x_train = np.asarray(new_x_train)
new_y_train = np.asarray(new_y_train)
print(new_x_train.shape)
print(new_y_train.shape)

(199200, 32, 32, 3)
(199200,)


In [ ]:
# Convert class vectors to binary class matrices.

new_y_train = keras.utils.to_categorical(new_y_train, 4)


In [ ]:
# define model
regularizer=keras.regularizers.l2(1e-3)
input = keras.layers.Input(shape=new_x_train[0].shape)

c1 = keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='elu', kernel_regularizer=regularizer)(input)
c2 = keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='elu', kernel_regularizer=regularizer)(c1)
m1 = keras.layers.MaxPooling2D()(c2)
c3 = keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='elu', kernel_regularizer=regularizer)(m1)
c4 = keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='elu', kernel_regularizer=regularizer)(c3)
m2 = keras.layers.MaxPooling2D()(c4)
f = keras.layers.Flatten(input_shape=x_train[0].shape)(m2)
dropout = keras.layers.Dropout(0.5)(f)
d1 = keras.layers.Dense(units=512, activation='elu', kernel_regularizer=regularizer)(dropout)
output = keras.layers.Dense(units=4, activation='softmax')(d1)

rotation_model = keras.Model(input, output)
rotation_model.summary()

Model: "model_60"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_43 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_168 (Conv2D)          (None, 30, 30, 128)       3584      
_________________________________________________________________
conv2d_169 (Conv2D)          (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d_84 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_170 (Conv2D)          (None, 12, 12, 128)       147584    
_________________________________________________________________
conv2d_171 (Conv2D)          (None, 10, 10, 128)       147584    
_________________________________________________________________
max_pooling2d_85 (MaxPooling (None, 5, 5, 128)         0  

In [ ]:
 # compile model
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)
rotation_model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

# train model
history_rotation = rotation_model.fit(new_x_train, new_y_train,
                    batch_size=256,
                    epochs=10,
                    shuffle=True,
                    validation_split = 0.2,

                    )

Epoch 1/10
623/623 [==============================] - 70s 111ms/step - loss: 5.5480 - accuracy: 0.3386 - val_loss: 1.9715 - val_accuracy: 0.5462
Epoch 2/10
623/623 [==============================] - 69s 110ms/step - loss: 1.8744 - accuracy: 0.5569 - val_loss: 1.5530 - val_accuracy: 0.6254
Epoch 3/10
623/623 [==============================] - 69s 110ms/step - loss: 1.4997 - accuracy: 0.6227 - val_loss: 1.2803 - val_accuracy: 0.6720
Epoch 4/10
623/623 [==============================] - 69s 110ms/step - loss: 1.2608 - accuracy: 0.6605 - val_loss: 1.0851 - val_accuracy: 0.7074
Epoch 5/10
594/623 [===========================>..] - ETA: 2s - loss: 1.1162 - accuracy: 0.6876

In [ ]:
# rotation_model.summary()
# rotation_model.layers.pop()
# ratate_model.layers.pop()
# rotation_model.summary()



In [ ]:
output_clf = keras.layers.Dense(units=num_classes, activation='softmax')(rotation_model.layers[-2].output)
clf_model = keras.Model(input, output_clf)
clf_model.summary()

In [ ]:
# output_clf = keras.layers.Dense(units=num_classes, activation='softmax')(d1)

# clf_model = keras.Model(input, output_clf)
# clf_model.summary()

In [ ]:
 # compile model
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
clf_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


# train model
history_clf = clf_model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=10,
                    validation_data=(x_test, y_test),
                    
                    )